# XGboost

In [1]:
import pandas as pd
import numpy as np
import os

import json

from sklearn.model_selection  import train_test_split

### 설정된 값들

In [2]:
DATA_IN_PATH = './data_in/'
DATA_OUT_PATH = './data_out/'

TRAIN_Q1_DATA_FILE = 'train_q1.npy'
TRAIN_Q2_DATA_FILE = 'train_q2.npy'
TRAIN_LABEL_DATA_FILE = 'train_label.npy'


# 훈련 데이터 가져오는 부분이다.
train_q1_data = np.load(open(DATA_IN_PATH + TRAIN_Q1_DATA_FILE, 'rb'))
train_q2_data = np.load(open(DATA_IN_PATH + TRAIN_Q2_DATA_FILE, 'rb'))
train_labels = np.load(open(DATA_IN_PATH + TRAIN_LABEL_DATA_FILE, 'rb'))

In [3]:
train_input = np.stack((train_q1_data, train_q2_data), axis=1) 

In [4]:
print(train_input.shape)

(298526, 2, 31)


### 훈련 셋과 평가 셋 나누기¶

In [5]:
train_input, eval_input, train_label, eval_label = train_test_split(train_input, train_labels, test_size=0.2, random_state=4242)

In [6]:
import xgboost as xgb

### 모델 구성

In [7]:
train_data = xgb.DMatrix(train_input.sum(axis=1), label=train_label) # 학습 데이터 읽어 오기
eval_data = xgb.DMatrix(eval_input.sum(axis=1), label=eval_label) # 평가 데이터 읽어 오기

data_list = [(train_data, 'train'), (eval_data, 'valid')]

In [8]:
params = {} # 인자를 통해 XGB모델에 넣어 주자 
params['objective'] = 'binary:logistic' # 로지스틱 예측을 통해서 
params['eval_metric'] = 'rmse' # root mean square error를 사용  

bst = xgb.train(params, train_data, num_boost_round = 1000, evals = data_list, early_stopping_rounds=10)

[15:35:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[0]	train-rmse:0.483759	valid-rmse:0.484271
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 10 rounds.
[15:35:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[1]	train-rmse:0.473438	valid-rmse:0.474588
[15:35:42] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[2]	train-rmse:0.466677	valid-rmse:0.468063
[15:35:42] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[3]	train-rmse:0.46175	valid-rmse:0.463451
[15:35:42] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[4]	train-rmse:0.458099	valid-rmse:0.460085
[15:35:43] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126

[15:36:02] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 112 extra nodes, 0 pruned nodes, max_depth=6
[52]	train-rmse:0.427474	valid-rmse:0.43558
[15:36:02] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 110 extra nodes, 0 pruned nodes, max_depth=6
[53]	train-rmse:0.427243	valid-rmse:0.435483
[15:36:03] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[54]	train-rmse:0.42653	valid-rmse:0.434963
[15:36:03] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 122 extra nodes, 0 pruned nodes, max_depth=6
[55]	train-rmse:0.42615	valid-rmse:0.434772
[15:36:03] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 124 extra nodes, 0 pruned nodes, max_depth=6
[56]	train-rmse:0.425715	valid-rmse:0.434415
[15:36:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[57]	train-rmse:0.425231	valid-rmse:0.434173
[15:36:04] src/tree/updater_prune.cc:74: tree pruning e

[15:36:22] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 120 extra nodes, 0 pruned nodes, max_depth=6
[105]	train-rmse:0.415045	valid-rmse:0.429408
[15:36:23] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 36 extra nodes, 0 pruned nodes, max_depth=6
[106]	train-rmse:0.415008	valid-rmse:0.429391
[15:36:23] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 122 extra nodes, 0 pruned nodes, max_depth=6
[107]	train-rmse:0.414755	valid-rmse:0.429264
[15:36:24] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 0 pruned nodes, max_depth=6
[108]	train-rmse:0.414719	valid-rmse:0.42926
[15:36:24] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=6
[109]	train-rmse:0.414669	valid-rmse:0.42925
[15:36:24] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 110 extra nodes, 0 pruned nodes, max_depth=6
[110]	train-rmse:0.414412	valid-rmse:0.429136
[15:36:25] src/tree/updater_prune.cc:74: tree pruni

[15:36:43] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 98 extra nodes, 0 pruned nodes, max_depth=6
[157]	train-rmse:0.40757	valid-rmse:0.426982
[15:36:43] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 36 extra nodes, 0 pruned nodes, max_depth=6
[158]	train-rmse:0.407548	valid-rmse:0.426975
[15:36:43] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 0 pruned nodes, max_depth=6
[159]	train-rmse:0.407531	valid-rmse:0.426965
[15:36:44] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=6
[160]	train-rmse:0.407517	valid-rmse:0.426952
[15:36:44] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 34 extra nodes, 0 pruned nodes, max_depth=6
[161]	train-rmse:0.407499	valid-rmse:0.426934
[15:36:44] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 0 pruned nodes, max_depth=6
[162]	train-rmse:0.407486	valid-rmse:0.42692
[15:36:45] src/tree/updater_prune.cc:74: tree pruning 

[15:37:02] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 36 extra nodes, 0 pruned nodes, max_depth=6
[209]	train-rmse:0.400154	valid-rmse:0.424647
[15:37:03] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 122 extra nodes, 0 pruned nodes, max_depth=6
[210]	train-rmse:0.399882	valid-rmse:0.424568
[15:37:03] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 120 extra nodes, 0 pruned nodes, max_depth=6
[211]	train-rmse:0.399619	valid-rmse:0.424508
[15:37:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[212]	train-rmse:0.399395	valid-rmse:0.424419
[15:37:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 106 extra nodes, 0 pruned nodes, max_depth=6
[213]	train-rmse:0.39921	valid-rmse:0.424417
[15:37:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 120 extra nodes, 0 pruned nodes, max_depth=6
[214]	train-rmse:0.398949	valid-rmse:0.424376
[15:37:05] src/tree/updater_prune.cc:74: tree pr

[15:37:22] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nodes, 0 pruned nodes, max_depth=6
[261]	train-rmse:0.393499	valid-rmse:0.423019
[15:37:23] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 88 extra nodes, 0 pruned nodes, max_depth=6
[262]	train-rmse:0.393368	valid-rmse:0.423006
[15:37:23] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 100 extra nodes, 0 pruned nodes, max_depth=6
[263]	train-rmse:0.393158	valid-rmse:0.422979
[15:37:24] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 0 pruned nodes, max_depth=6
[264]	train-rmse:0.392956	valid-rmse:0.422977
[15:37:24] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 72 extra nodes, 0 pruned nodes, max_depth=6
[265]	train-rmse:0.392893	valid-rmse:0.42296
[15:37:24] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 0 pruned nodes, max_depth=6
[266]	train-rmse:0.392869	valid-rmse:0.422953
[15:37:25] src/tree/updater_prune.cc:74: tree pruni

[15:37:42] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=6
[313]	train-rmse:0.38778	valid-rmse:0.42183
[15:37:43] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 0 pruned nodes, max_depth=6
[314]	train-rmse:0.387773	valid-rmse:0.421828
[15:37:43] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 0 pruned nodes, max_depth=6
[315]	train-rmse:0.387766	valid-rmse:0.421817
[15:37:44] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 32 extra nodes, 0 pruned nodes, max_depth=6
[316]	train-rmse:0.387743	valid-rmse:0.421804
[15:37:44] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 76 extra nodes, 0 pruned nodes, max_depth=6
[317]	train-rmse:0.387699	valid-rmse:0.421802
[15:37:44] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nodes, 0 pruned nodes, max_depth=6
[318]	train-rmse:0.387686	valid-rmse:0.421794
[15:37:45] src/tree/updater_prune.cc:74: tree pruning 

[15:38:03] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 120 extra nodes, 0 pruned nodes, max_depth=6
[365]	train-rmse:0.380183	valid-rmse:0.420198
[15:38:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 120 extra nodes, 0 pruned nodes, max_depth=6
[366]	train-rmse:0.380005	valid-rmse:0.420192
[15:38:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nodes, 0 pruned nodes, max_depth=6
[367]	train-rmse:0.37999	valid-rmse:0.420199
[15:38:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 92 extra nodes, 0 pruned nodes, max_depth=6
[368]	train-rmse:0.379915	valid-rmse:0.420171
[15:38:05] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 96 extra nodes, 0 pruned nodes, max_depth=6
[369]	train-rmse:0.379765	valid-rmse:0.420162
[15:38:05] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 36 extra nodes, 0 pruned nodes, max_depth=6
[370]	train-rmse:0.379755	valid-rmse:0.420154
[15:38:05] src/tree/updater_prune.cc:74: tree pruni

[15:38:23] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 0 pruned nodes, max_depth=6
[417]	train-rmse:0.374265	valid-rmse:0.419195
[15:38:23] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 116 extra nodes, 0 pruned nodes, max_depth=6
[418]	train-rmse:0.374104	valid-rmse:0.419155
[15:38:24] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 52 extra nodes, 0 pruned nodes, max_depth=6
[419]	train-rmse:0.374083	valid-rmse:0.41915
[15:38:24] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 116 extra nodes, 0 pruned nodes, max_depth=6
[420]	train-rmse:0.373901	valid-rmse:0.41912
[15:38:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 80 extra nodes, 0 pruned nodes, max_depth=6
[421]	train-rmse:0.373843	valid-rmse:0.419127
[15:38:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 106 extra nodes, 0 pruned nodes, max_depth=6
[422]	train-rmse:0.3737	valid-rmse:0.419117
[15:38:25] src/tree/updater_prune.cc:74: tree pruning

[15:38:43] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 116 extra nodes, 0 pruned nodes, max_depth=6
[469]	train-rmse:0.369187	valid-rmse:0.418619
[15:38:43] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 122 extra nodes, 0 pruned nodes, max_depth=6
[470]	train-rmse:0.368999	valid-rmse:0.41858
[15:38:44] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 88 extra nodes, 0 pruned nodes, max_depth=6
[471]	train-rmse:0.368883	valid-rmse:0.418613
[15:38:44] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 82 extra nodes, 0 pruned nodes, max_depth=6
[472]	train-rmse:0.368782	valid-rmse:0.418615
[15:38:44] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 64 extra nodes, 0 pruned nodes, max_depth=6
[473]	train-rmse:0.368762	valid-rmse:0.418619
[15:38:45] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 84 extra nodes, 0 pruned nodes, max_depth=6
[474]	train-rmse:0.368688	valid-rmse:0.418646
[15:38:45] src/tree/updater_prune.cc:74: tree pruni

### 테스트 데이터 가져오기

In [9]:
TEST_Q1_DATA_FILE = 'test_q1.npy'
TEST_Q2_DATA_FILE = 'test_q2.npy'
TEST_ID_DATA_FILE = 'test_id.npy'

test_q1_data = np.load(open(DATA_IN_PATH + TEST_Q1_DATA_FILE, 'rb'))
test_q2_data = np.load(open(DATA_IN_PATH + TEST_Q2_DATA_FILE, 'rb'))
test_id_data = np.load(open(DATA_IN_PATH + TEST_ID_DATA_FILE, 'rb'))

### 예측하기

In [12]:
test_input = np.stack((test_q1_data, test_q2_data), axis=1) 
test_data = xgb.DMatrix(test_input.sum(axis=1))
test_predict = bst.predict(test_data)

In [13]:
if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)
    
output = pd.DataFrame({'test_id': test_id_data, 'is_duplicate': test_predict})
output.to_csv(DATA_OUT_PATH + 'simple_xgb.csv', index=False)